# PyDipAPI - Async API Tutorial

Dieses Notebook demonstriert die asynchronen Features von PyDipAPI fuer bessere Performance.

## Was Sie lernen werden:
1. **AsyncDipAnfrage** - Asynchroner Client
2. **Context Managers** - Ressourcen-Management
3. **Performance-Vergleich** - Sync vs. Async
4. **Parallele Anfragen** - Mehrere Requests gleichzeitig
5. **Error Handling** - Async-spezifische Fehlerbehandlung
6. **Praktische Use Cases** - Reale Anwendungsfaelle
7. **Uebungsaufgaben** - Ihr Wissen testen

In [ ]:
import sys
import os
import asyncio
import time
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage
from pydipapi.async_api import AsyncDipAnfrage

# API-Konfiguration
API_KEY = "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN"

if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    sync_client = DipAnfrage(api_key=API_KEY)
    print("Async API Tutorial bereit!")
else:
    print("BITTE ERSETZEN SIE DEN PLATZHALTER MIT IHREM ECHTEN API-SCHLUESSEL!")
    print("Den API-Schluessel erhalten Sie unter: https://dip.bundestag.de/api/")

## AsyncDipAnfrage - Grundlagen

Der AsyncDipAnfrage Client erlaubt parallele API-Anfragen fuer bessere Performance.

In [ ]:
# Async Client testen
async def test_async_basic():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Teste AsyncDipAnfrage...")
        
        async with AsyncDipAnfrage(api_key=API_KEY) as async_client:
            try:
                persons = await async_client.get_person(anzahl=3)
                print(f"{len(persons)} Personen asynchron geladen")
                
                if persons:
                    person = persons[0]
                    name = f"{person.get('vorname', '')} {person.get('nachname', '')}"
                    print(f"Erste Person: {name.strip()}")
                    
            except Exception as e:
                print(f"Fehler: {e}")
        
        print("Async Client ordnungsgemaess geschlossen")
    else:
        print("Bitte API-Schluessel konfigurieren.")

# Async Funktion ausfuehren
await test_async_basic()

## Performance-Vergleich: Sync vs. Async

Vergleichen Sie die Performance von synchronen und asynchronen Anfragen.

In [ ]:
# Performance-Vergleich
async def performance_test():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        num_requests = 4
        print(f"Performance-Test mit {num_requests} Anfragen")
        print("=" * 50)
        
        # Synchrone Anfragen (sequenziell)
        print(f"Synchrone Anfragen...")
        start_time = time.time()
        
        sync_results = []
        for i in range(num_requests):
            try:
                result = sync_client.get_person(anzahl=2)
                sync_results.append(len(result) if result else 0)
            except Exception as e:
                print(f"Sync Fehler {i+1}: {e}")
                sync_results.append(0)
        
        sync_time = time.time() - start_time
        print(f"Sync Zeit: {sync_time:.2f} Sekunden")
        
        # Asynchrone Anfragen (parallel)
        print(f"\nAsynchrone Anfragen...")
        start_time = time.time()
        
        async def fetch_async(client, request_id):
            try:
                result = await client.get_person(anzahl=2)
                return len(result) if result else 0
            except Exception as e:
                print(f"Async Fehler {request_id}: {e}")
                return 0
        
        async with AsyncDipAnfrage(api_key=API_KEY) as async_client:
            tasks = [fetch_async(async_client, i+1) for i in range(num_requests)]
            async_results = await asyncio.gather(*tasks)
        
        async_time = time.time() - start_time
        print(f"Async Zeit: {async_time:.2f} Sekunden")
        
        # Ergebnisse
        print(f"\nPerformance-Gewinn:")
        if async_time > 0:
            speedup = sync_time / async_time
            print(f"- Async ist {speedup:.1f}x schneller!")
            print(f"- Zeitersparnis: {sync_time - async_time:.2f} Sekunden")
        
        print(f"\nErgebnisse:")
        print(f"- Sync: {sync_results}")
        print(f"- Async: {async_results}")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await performance_test()

## Erweiterte Async-Patterns

Verschiedene Datentypen parallel abrufen.

In [ ]:
# Verschiedene Datentypen parallel abrufen
async def mixed_data_fetching():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Verschiedene Datentypen parallel abrufen...")
        
        async with AsyncDipAnfrage(api_key=API_KEY) as client:
            # Alle Anfragen parallel starten
            tasks = [
                client.get_person(anzahl=3),
                client.get_drucksache(anzahl=3),
                client.get_aktivitaet(anzahl=3)
            ]
            
            try:
                persons, documents, activities = await asyncio.gather(*tasks)
                
                print(f"Parallel geladen:")
                print(f"  - Personen: {len(persons) if persons else 0}")
                print(f"  - Dokumente: {len(documents) if documents else 0}")
                print(f"  - Aktivitaeten: {len(activities) if activities else 0}")
                
            except Exception as e:
                print(f"Fehler beim parallelen Laden: {e}")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await mixed_data_fetching()

## Error Handling in Async Context

Robuste Fehlerbehandlung in asynchronen Kontexten.

In [ ]:
# Robuste Fehlerbehandlung
async def robust_async_fetching():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Teste robuste Fehlerbehandlung...")
        
        async def safe_fetch(client, fetch_func, name):
            try:
                result = await fetch_func()
                print(f"{name}: {len(result) if result else 0} Elemente")
                return result
            except Exception as e:
                print(f"{name} Fehler: {e}")
                return []
        
        async with AsyncDipAnfrage(api_key=API_KEY) as client:
            # Verschiedene Anfragen mit individueller Fehlerbehandlung
            tasks = [
                safe_fetch(client, lambda: client.get_person(anzahl=2), "Personen"),
                safe_fetch(client, lambda: client.get_drucksache(anzahl=2), "Dokumente"),
                safe_fetch(client, lambda: client.get_aktivitaet(anzahl=2), "Aktivitaeten")
            ]
            
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            successful_requests = sum(1 for r in results if not isinstance(r, Exception))
            print(f"\nErfolgreiche Anfragen: {successful_requests}/{len(tasks)}")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await robust_async_fetching()

## Praktische Use Cases

### Use Case 1: Parallele Datenabfrage fuer Dashboard

Laden Sie verschiedene Datentypen parallel, um ein Dashboard zu befuellen.

In [ ]:
# Use Case: Dashboard-Daten parallel laden
async def load_dashboard_data():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Lade Dashboard-Daten parallel...")
        
        async with AsyncDipAnfrage(api_key=API_KEY) as client:
            start_time = time.time()
            
            # Alle Daten parallel laden
            persons_task = client.get_person(anzahl=10)
            documents_task = client.get_drucksache(anzahl=10)
            activities_task = client.get_aktivitaet(anzahl=10)
            
            persons, documents, activities = await asyncio.gather(
                persons_task,
                documents_task,
                activities_task
            )
            
            elapsed = time.time() - start_time
            
            print(f"\nDashboard-Daten geladen in {elapsed:.2f}s:")
            print(f"  - Personen: {len(persons) if persons else 0}")
            print(f"  - Dokumente: {len(documents) if documents else 0}")
            print(f"  - Aktivitaeten: {len(activities) if activities else 0}")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await load_dashboard_data()

### Use Case 2: Batch-Verarbeitung mit Async

Verarbeiten Sie mehrere IDs parallel mit asynchronen Batch-Anfragen.

In [ ]:
# Use Case: Batch-Verarbeitung
async def batch_processing():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Batch-Verarbeitung mit Async...")
        
        # Zuerst einige IDs sammeln
        sync_client = DipAnfrage(api_key=API_KEY)
        persons = sync_client.get_person(anzahl=5)
        person_ids = [p.get('id') for p in persons if p.get('id')][:3]
        
        if person_ids:
            print(f"Verarbeite {len(person_ids)} Personen-IDs parallel...")
            
            async with AsyncDipAnfrage(api_key=API_KEY) as client:
                start_time = time.time()
                
                # Alle IDs parallel abrufen
                tasks = [client.get_person_id(pid) for pid in person_ids]
                results = await asyncio.gather(*tasks)
                
                elapsed = time.time() - start_time
                
                successful = sum(1 for r in results if r)
                print(f"\n{successful}/{len(person_ids)} Personen erfolgreich geladen in {elapsed:.2f}s")
        else:
            print("Keine IDs gefunden")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await batch_processing()

### Use Case 3: Monitoring mit periodischen Async-Anfragen

Ueberwachen Sie aktuelle Aktivitaeten mit periodischen asynchronen Anfragen.

In [ ]:
# Use Case: Monitoring
async def monitor_activities():
    if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
        print("Monitoring: Aktuelle Aktivitaeten...")
        
        async with AsyncDipAnfrage(api_key=API_KEY) as client:
            # Mehrere Monitoring-Zyklen
            for cycle in range(2):
                print(f"\nZyklus {cycle + 1}:")
                
                try:
                    activities = await client.get_recent_activities(days=1, anzahl=5)
                    print(f"  Gefunden: {len(activities) if activities else 0} Aktivitaeten")
                    
                    if activities:
                        from collections import Counter
                        types = [a.get('aktivitaetstyp', 'Unbekannt') for a in activities]
                        type_counts = Counter(types)
                        print(f"  Aktivitaetstypen: {dict(type_counts)}")
                    
                    # Warten vor naechstem Zyklus
                    if cycle < 1:
                        await asyncio.sleep(1)
                except Exception as e:
                    print(f"  Fehler: {e}")
        
        print("\nMonitoring abgeschlossen")
    else:
        print("Bitte API-Schluessel konfigurieren.")

await monitor_activities()

## Uebungsaufgaben

Testen Sie Ihr Wissen mit diesen praktischen Aufgaben:

### Aufgabe 1: Performance-Vergleich erweitern

Schreiben Sie Code, der:
1. 10 synchrone Anfragen misst
2. 10 asynchrone Anfragen misst
3. Den Performance-Gewinn berechnet und ausgibt

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie time.time() und asyncio.gather()


### Aufgabe 2: Parallele Suche

Schreiben Sie Code, der:
1. Parallel nach 3 verschiedenen Suchbegriffen sucht
2. Die Ergebnisse kombiniert
3. Die Gesamtzahl der gefundenen Dokumente ausgibt

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie search_documents() mit asyncio.gather()


### Aufgabe 3: Robuste Batch-Verarbeitung

Schreiben Sie Code, der:
1. Eine Liste von Person-IDs parallel abruft
2. Fehlerhafte Anfragen abfaengt
3. Eine Statistik ueber erfolgreiche/fehlgeschlagene Anfragen ausgibt

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie try/except in async Funktionen und return_exceptions=True


## Zusammenfassung

In diesem Notebook haben Sie gelernt:

- **AsyncDipAnfrage**: Asynchroner Client fuer parallele Anfragen
- **Context Manager**: Automatisches Ressourcen-Management
- **Performance**: Erhebliche Geschwindigkeitsverbesserungen
- **Parallelitaet**: Mehrere Anfragen gleichzeitig
- **Error Handling**: Robuste Fehlerbehandlung
- **Praktische Use Cases**: Reale Anwendungsfaelle
- **Uebungsaufgaben**: Ihr Wissen testen

### Wann Async verwenden:

- **Viele API-Anfragen**: Parallele Verarbeitung spart Zeit
- **I/O-intensive Operationen**: Waehrend Netzwerk-Wartezeiten
- **Real-time Anwendungen**: Fuer bessere Responsivitaet
- **Batch-Verarbeitung**: Mehrere IDs gleichzeitig abrufen

### Naechste Schritte

Weiter mit **Notebook 06**: Datenvisualisierung fuer professionelle Analysen!